In [ ]:
import json
import pandas as pd
import cobra
from cobra.io import load_matlab_model, load_json_model, save_json_model, load_model
from bioservices import UniProt
from tqdm import tqdm
from p_tqdm import p_map
import pickle

## Access StringDB through sql

In [2]:
# import sqlalchemy as db

In [ ]:
# def get_interaction():
#     """ query data from the network table """
#     conn = None
#     try:
#         params = config()
#         conn = psycopg2.connect(**params)
#         cur = conn.cursor()
#         cur.execute("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")
#         print("The number of parts: ", cur.rowcount)
#         row = cur.fetchone()

#         while row is not None:
#             print(row)
#             row = cur.fetchone()

#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [8]:
# engine = db.create_engine('postgresql://user:password:/Mounts/rbg-storage1/datasets/STRING/network_schema.v11.5.sql')

ValueError: invalid literal for int() with base 10: 'memory:'

In [ ]:
# connection = engine.connect()

In [4]:
# metadata = db.MetaData()
# node_node_links = db.Table('node_node_links', metadata, autoload=True, autoload_with=engine)

OperationalError: (psycopg2.OperationalError) connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# # Print full table metadata
# print(repr(metadata.tables['node_node_links']))

In [ ]:
# # Equivalent to 'SELECT * FROM census'
# query = db.select([node_node_links]) 

In [ ]:
# ResultProxy = connection.execute(query)

In [ ]:
# ResultSet = ResultProxy.fetchone()

## Get String E. coli DB

In [2]:
prots = pd.read_csv("/Mounts/rbg-storage1/datasets/STRING/e_coli/511145.protein.info.v11.5.txt", sep = "\t")
prots["#string_protein_id"] = prots["#string_protein_id"].str.replace("511145\.", "", regex=True)
links = pd.read_csv("/Mounts/rbg-storage1/datasets/STRING/e_coli/511145.protein.links.v11.5.txt", sep = " ")
links["protein1"] = links["protein1"].str.replace("511145\.", "", regex=True)
links["protein2"] = links["protein2"].str.replace("511145\.", "", regex=True)

## Get BiGG Genes

In [3]:
model = load_matlab_model(f"/Mounts/rbg-storage1/datasets/Metabo/BiGG/iML1515.mat")

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model iML1515. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [4]:
all_genes = set()
for pathway in model.groups:
    for reaction in pathway.members:
        for gene in reaction.genes:
            all_genes.add(gene.id)

In [5]:
len(all_genes)

1516

In [6]:
ds = json.load(open("/Mounts/rbg-storage1/datasets/Metabo/datasets/iML1515_dataset.json", "rb"))

In [7]:
all_big_ids = set()
for sample in ds:
    if 'proteins' in sample:
        for prot in sample['proteins']:
            if 'bigg_gene_id' in prot:
                all_big_ids.add(prot["bigg_gene_id"])
            else:
                print("Not in prot")

## All except 3 genes from BiGG are in PPI

In [8]:
len(all_big_ids)

1516

In [9]:
ppi_genes = set(prots["#string_protein_id"].values)

In [10]:
len(ppi_genes)

4127

In [11]:
len(ppi_genes - all_big_ids), len(ppi_genes) - len(all_big_ids)

(2614, 2611)

In [12]:
all_big_ids - ppi_genes

{'b2092', 'b4104', 's0001'}

## Get drug targets from Stokes dataset

In [13]:
stokes_targets = pd.read_csv("/Mounts/rbg-storage1/datasets/Metabo/antibiotics/stokes2019_dataset_with_chembl_ids_and_targets_uniprot_sql.csv")

In [14]:
stokes_targets.head()

,smiles,A0JN73,A1XG20,A5F384,A5X5Y0,A5Z1V5,A6NHL2,B2RXH2,C4YTQ8,ENSG00000284190,...,Q9Y6L6,Q9YQ12,Q9Z1B7,Q9Z1E3,Q9Z1J7,Q9Z1Q2,Q9Z2V5,Q9Z2V6,Q9ZFE4,T1VXA1
0,C=CCC1(O)CCC2C3CCC4=CC(=O)CCC4=C3C=CC21C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Oc1c(Cl)cc(Cl)cc1Sc1cc(Cl)cc(Cl)c1O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cc1c[nH]cn1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,COC1=CC(=O)OC(C=Cc2ccccc2)C1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
u = UniProt(verbose=False)

In [16]:
u.mapping("UniProtKB_AC-ID", "Gene_Name", query='A0JN73')

{'results': [{'from': 'A0JN73', 'to': 'TOP3A'}]}

In [18]:
def get_mapping(query):
    try:
        return (query, u.mapping("UniProtKB_AC-ID", "KEGG", query=query))
    except:
        return (query, None)

In [19]:
# query2mapping = p_map(get_mapping, [query for query in stokes_targets.columns if query != 'smiles'])

In [20]:
query2mapping = []
for query in tqdm(stokes_targets.columns):
    if query != 'smiles':
        query2mapping.append(get_mapping(query))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 977/977 [08:56<00:00,  1.82it/s]


In [21]:
uniprot2kegg = {}
for query, mapping in query2mapping:
    if mapping is not None and len(mapping['results']) > 0:
        uniprot2kegg[query] = [result['to'] for result in mapping['results']]

In [22]:
len(uniprot2kegg)

892

In [ ]:
stokes_json = json.load(open("/Mounts/rbg-storage1/datasets/Metabo/antibiotics/stokes2019_dataset.json", "rb"))

In [31]:
uniprot2kegg_ecoli = {k: v[0][4:] for k,v in {k: [v_eco for v_eco in v if 'eco' in v_eco] for k,v in uniprot2kegg.items()}.items() if len(v) > 0}

In [ ]:
pickle.dump(uniprot2kegg_ecoli, open("uniprot2kegg_ecoli.pkl", "wb"))

In [43]:
len(set(prots["#string_protein_id"].values) - set(uniprot2kegg_ecoli.values())), len(set(prots["#string_protein_id"].values)), len(set(uniprot2kegg_ecoli.values()))

(4055, 4127, 72)

## Map Stokes dataset genes to BiGG model genes

In [ ]:
# distance to target

In [ ]:
path_distances = pickle.load(open("path_lengths.pkl", "rb"))

In [ ]:
pickle

In [ ]:
# normalized distance to target

In [ ]:
# hops to target